<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_30WMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-10-27 16:59:44


In [13]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [14]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
mypf = mypf[~mypf['Criteria'].isin(['V40','V40N'])]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ','ADANITRANS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [15]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [16]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [17]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 10
      #mid_window = 50
      long_window = 30
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1wk')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['30_WMA'] = round(stock_df['Close'].ewm(span=long_window, adjust=False).mean(),0)
      stock_df['Dev%_30'] = round((stock_df['Close'] - stock_df['30_WMA'])*100/stock_df['30_WMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):
    #date = (datetime.date.today() + datetime.timedelta(days=0))
    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] > df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] > 60) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] < 60) ), -1.0, df_prec_dev['Signal'])

    df_prec_dev['Position'] = df_prec_dev['Signal'].diff()

    df_pos = df_prec_dev[(df_prec_dev['Position'] >= 1) | (df_prec_dev['Position'] <= -1)]
    df_pos = df_pos[df_pos['Signal'] != 0]
    df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
    df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
    df_pos.drop('Prev_Signal', axis=1, inplace=True)

    return df_pos.tail(1)

def get_weekly_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_buy_sell = df_buy_sell.append(df_tmp)


    df_buy_sell['Position'] = df_buy_sell['Position'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()


In [18]:
df_weekly_buy_sell = get_weekly_buy_sell()
df_weekly_buy_sell

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2023-02-06 00:00:00+05:30,541.0,671.0,-19.37,AARTIIND.NS,1099.51,449.40,20.39,21.0,-1.0,Sell,VR
2023-02-06 00:00:00+05:30,1484.0,1577.0,-5.87,CLEAN.NS,2574.02,1264.99,17.35,46.0,-1.0,Sell,VR
2023-02-06 00:00:00+05:30,159.0,168.0,-5.45,CUB.NS,239.46,119.00,33.48,35.0,-1.0,Sell,AR
2023-02-06 00:00:00+05:30,142.0,135.0,5.56,ONGC.NS,191.85,127.53,11.74,62.0,1.0,Buy,NaN
2023-02-06 00:00:00+05:30,281.0,238.0,18.11,INDIANB.NS,426.05,242.17,16.08,61.0,1.0,Buy,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2023-10-23 00:00:00+05:30,1606.0,1634.0,-1.74,PVRINOX.NS,1899.00,1372.60,16.98,47.0,-1.0,Sell,NaN
2023-10-23 00:00:00+05:30,1988.0,2056.0,-3.31,DEEPAKNTR.NS,2875.84,1773.39,12.10,45.0,-1.0,Sell,NaN
2023-10-23 00:00:00+05:30,3412.0,3526.0,-3.23,DIVISLAB.NS,5246.19,2742.98,24.39,44.0,-1.0,Sell,NaN


In [19]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Sell'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2023-10-16 00:00:00+05:30,231.0,254.0,-9.07,BIOCON.NS,475.61,196.40,17.59,40.0,-1.0,Sell,NaN
2023-10-23 00:00:00+05:30,732.0,760.0,-3.68,JSWSTEEL.NS,820.65,655.04,11.75,43.0,-1.0,Sell,NaN
2023-10-23 00:00:00+05:30,87.0,88.0,-1.25,IOC.NS,96.33,66.42,30.83,47.0,-1.0,Sell,NaN
2023-10-23 00:00:00+05:30,820.0,867.0,-5.39,WESTLIFE.NS,985.05,665.48,23.26,41.0,-1.0,Sell,AR
2023-10-23 00:00:00+05:30,607.0,629.0,-3.55,LICI.NS,816.41,529.23,14.63,42.0,-1.0,Sell,V200
2023-10-23 00:00:00+05:30,169.0,174.0,-3.10,INDUSTOWER.NS,300.31,137.00,23.07,47.0,-1.0,Sell,NaN
2023-10-23 00:00:00+05:30,1878.0,1938.0,-3.10,ACC.NS,2592.95,1650.30,13.80,45.0,-1.0,Sell,NaN
2023-10-23 00:00:00+05:30,1608.0,1620.0,-0.74,TATACOMM.NS,1932.75,1169.40,37.50,48.0,-1.0,Sell,AR
2023-10-23 00:00:00+05:30,4830.0,4867.0,-0.76,APOLLOHOSP.NS,5643.71,4186.58,15.37,48.0,-1.0,Sell,NaN


In [20]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Buy'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2023-07-17 00:00:00+05:30,340.0,312.0,9.00,VAIBHAVGBL.NS,962.57,271.79,25.12,64.0,1.0,Buy,VR
2023-07-24 00:00:00+05:30,245.0,211.0,16.18,ZEEL.NS,471.81,176.65,38.78,67.0,1.0,Buy,NaN
2023-07-24 00:00:00+05:30,450.0,408.0,10.31,EMAMILTD.NS,585.26,351.65,27.98,66.0,1.0,Buy,V200
2023-07-24 00:00:00+05:30,403.0,376.0,7.15,ABSLAMC.NS,663.47,307.22,31.14,64.0,1.0,Buy,AR
2023-07-24 00:00:00+05:30,405.0,369.0,9.80,LICHSGFIN.NS,492.99,314.23,28.94,65.0,1.0,Buy,AR
2023-08-07 00:00:00+05:30,801.0,724.0,10.64,ADANIPORTS.NS,931.93,495.52,61.66,64.0,1.0,Buy,NaN
2023-08-07 00:00:00+05:30,1648.0,1328.0,24.10,GLAND.NS,4287.20,922.25,78.69,69.0,1.0,Buy,VR
2023-08-07 00:00:00+05:30,459.0,428.0,7.35,HINDALCO.NS,613.04,385.88,19.07,61.0,1.0,Buy,NaN
2023-08-07 00:00:00+05:30,16.0,15.0,10.00,HATHWAY.NS,48.95,12.95,27.41,63.0,1.0,Buy,VR


In [21]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [22]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)